In [7]:
import os, glob
import torch
import pandas as pd
import numpy as np
import pickle
import joblib
import argparse

In [3]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.utils import shuffle
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import train_test_split, GridSearchCV

In [15]:
from train_RF import get_name, create_dataset, load_model, setup, sample

In [5]:
target = 'y' #TODO: da provare anche con hat

In [9]:
import sys
sys.argv = [
    'train_RF.py',                   # fittizio
    '../../input_imgs',
    '../../output/'         
    
]
parser = argparse.ArgumentParser(description='Compress a directory of images using the RecoEncoder')
parser.add_argument('--gpu', type=int, default=None, help='GPU index')
parser.add_argument('--imgs_path', type=str, default='../../real_vs_fake/real-vs-fake', help='Input directory')
parser.add_argument('input_path', type=str, default='../../input_imgs', help='Input directory')
parser.add_argument('bin_path', type=str, default='../../JPEGAI_output/', help='Save directory')
parser.add_argument('--set_target_bpp', type=int, default=1, help='Set the target bpp '
                                                                    '(multiplied by 100)')
parser.add_argument('--models_dir_name', type=str, default='../../jpeg-ai-reference-software/models', help='Directory name for the '
                                                                                'models used in the encoder-decoder'
                                                                                'pipeline')
#Arguments for training
parser.add_argument('--num_samples', type=int, default=None, help='Number of samples to process')
parser.add_argument("--train_csv",default="../../train.csv" , help="Path to dataset's csv file")
parser.add_argument("--test_csv", default="../../test.csv", help="Path to test's csv file")
parser.add_argument("-t", "--target",default="y", help="y_hat if quantized latent, else y")
parser.add_argument("--models_save_dir", default="/data/lesc/users/rustichini/thesis/models_saved", help="Directory to save models")
args = parser.parse_args()

coder = setup(args)

Codec/Encoder: name=JAI, version=DIS


In [13]:
imgs_dir = args.imgs_path
target = args.target

df_test = pd.read_csv(args.test_csv)
n_test_samples = 10
df_test = sample(df_test,n_test_samples)
X_test, y_test = create_dataset(coder, df_test,imgs_dir, target)

  0%|          | 0/10 [00:00<?, ?it/s]

100%|██████████| 10/10 [01:04<00:00,  6.49s/it]


In [21]:
model_path = "/data/lesc/users/rustichini/thesis/models_saved/10_samples/y/RF_1_estimators.joblib"
model = load_model(model_path)
y_pred = model.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00         5
           1       0.44      0.80      0.57         5

    accuracy                           0.40        10
   macro avg       0.22      0.40      0.29        10
weighted avg       0.22      0.40      0.29        10

